In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install faiss-cpu pdfplumber docling -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.9/276.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.9 MB/s eta 0:00:00
   ━━

In [ ]:
import os
import json
import faiss
import numpy as np
import pdfplumber
from typing import List, Dict
from docx import Document
from sentence_transformers import SentenceTransformer
from docling.document_converter import DocumentConverter
from docling_core.types.doc import DocItemLabel

In [ ]:
BATCH_SIZE = 4
MAX_CHUNK_SIZE = 1000
DOC_FILEPATH = '/content/drive/MyDrive/hust_docs'
OUT_DIR = '/content/drive/MyDrive/rag'
EMBED_MODEL = 'BAAI/bge-m3'

In [ ]:
def normalize_token(w: str) -> str:
    return ''.join(c for c in w.lower() if c.isalnum())


def pdf_to_dict(pdf_path: str) -> set:
    words = set()
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            for w in text.split():
                w = normalize_token(w)
                if 2 <= len(w) <= 8:
                    words.add(w)
    return words


def docx_to_dict(docx_path: str) -> set:
    words = set()
    doc = Document(docx_path)
    for para in doc.paragraphs:
        text = para.text or ""
        for w in text.split():
            w = normalize_token(w)
            if 2 <= len(w) <= 8:
                words.add(w)
    return words


def repair_raw(line: str, dictionary: set) -> str:
    tokens = line.split(' ')
    repaired = []
    i = 0

    while i < len(tokens):
        t0 = normalize_token(tokens[i])

        if i + 3 < len(tokens):
            merged4 = normalize_token(tokens[i] + tokens[i + 1] + tokens[i + 2] + tokens[i + 3])
            if merged4 in dictionary:
                repaired.append(tokens[i] + tokens[i + 1] + tokens[i + 2] + tokens[i + 3])
                i += 4
                continue

        if i + 2 < len(tokens):
            merged3 = normalize_token(tokens[i] + tokens[i + 1] + tokens[i + 2])
            if merged3 in dictionary:
                repaired.append(tokens[i] + tokens[i + 1] + tokens[i + 2])
                i += 3
                continue

        if i + 1 < len(tokens):
            merged2 = normalize_token(tokens[i] + tokens[i + 1])
            if merged2 in dictionary:
                repaired.append(tokens[i] + tokens[i + 1])
                i += 2
                continue

        repaired.append(tokens[i])
        i += 1

    return " ".join(repaired)


def repair(text: str, dictionary: set) -> str:
    out = []
    for line in text.splitlines(keepends=True):
        newline = "\n" if line.endswith("\n") else ""
        content = line.rstrip("\n")
        out.append(repair_raw(content, dictionary) + newline)
    return "".join(out)

def iter_blocks(document, max_chunk_size: int):
    blocks = []

    current_section = "ROOT"
    current_content = []
    current_length = 0

    def flush():
        nonlocal current_content, current_length
        if current_content:
            blocks.append({
                "type": "section",
                "section": current_section,
                "content": "\n\n".join(current_content)
            })
            current_content = []
            current_length = 0

    for item, stack in document.iterate_items():
        label = item.label

        if label in (DocItemLabel.TITLE, DocItemLabel.SECTION_HEADER):
            text = item.text.strip()
            if text and stack <= 0:
                flush()
                current_section = text
                current_content = [text]
                current_length = len(text)
            continue

        if label in (DocItemLabel.PARAGRAPH, DocItemLabel.TEXT, DocItemLabel.LIST_ITEM):
            text = item.text.strip()
            if not text:
                continue

            text_len = len(text) + (2 if current_content else 0)

            if current_length + text_len > max_chunk_size:
                flush()

            current_content.append(text)
            current_length += text_len
            continue

        if label == DocItemLabel.TABLE:
            table_md = item.export_to_markdown(document)
            flush()
            blocks.append({
                "type": "table",
                "section": current_section,
                "content": table_md
            })
            continue

    flush()
    return blocks

def convert_file(file_path, max_chunk_size = MAX_CHUNK_SIZE) -> List[Dict]:
    try:
        ext = os.path.splitext(file_path)[1].lower()
        converter = DocumentConverter()

        print(f"Converting: {file_path}")
        result = converter.convert(file_path)

        if ext == ".pdf":
            dictionary = pdf_to_dict(file_path)
        elif ext == ".docx":
            dictionary = docx_to_dict(file_path)
        else:
            return []

        blocks = iter_blocks(result.document, max_chunk_size)
        docs = []

        for b in blocks:
            text = repair(b["content"], dictionary)
            metadata = {
                "source": os.path.basename(file_path),
                "section": repair(b["section"], dictionary),
                "type": b["type"]
            }

            docs.append({
                "text": text,
                "metadata": metadata
            })

        return docs

    except Exception as e:
        print(f"[ERROR] Failed processing: {file_path}")
        print(str(e))
        return []


def build_index(all_docs: List[Dict], embedder) -> faiss.Index:
    dim = embedder.get_sentence_embedding_dimension()
    index = faiss.IndexFlatIP(dim)

    texts = [d["text"] for d in all_docs]

    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i + BATCH_SIZE]
        emb = embedder.encode(
            batch,
            normalize_embeddings=True,
            show_progress_bar=False
        )
        index.add(np.asarray(emb, dtype="float32"))
        if i % (BATCH_SIZE * 10) == 0:
            print(f"Indexed {min(i + BATCH_SIZE, len(texts))}/{len(texts)}")

    return index

In [ ]:
docs: List[Dict] = []

for fn in sorted(os.listdir(DOC_FILEPATH)):
    if fn.lower().endswith((".pdf", ".docx")):
        docs.extend(convert_file(os.path.join(DOC_FILEPATH, fn), MAX_CHUNK_SIZE))

print(f"Total blocks: {len(docs)}")

embedder = SentenceTransformer(EMBED_MODEL)
index = build_index(docs, embedder)

os.makedirs(OUT_DIR, exist_ok=True)
faiss.write_index(index, os.path.join(OUT_DIR, "rag.index"))

with open(os.path.join(OUT_DIR, "metadata.json"), "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("Index and metadata saved")

Converting: /content/drive/MyDrive/hust_docs/01_1%202015%20TT%20Lien%20tich_QD%20danh%20gia%20QP-AN.pdf


[INFO] 2026-01-03 08:20:15,093 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:20:15,108 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:20:15,117 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:20:16,509 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2026-01-03 08:20:16,980 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:20:16,984 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:20:18,007 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:20:18,015 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:20:18,018 [RapidOCR] download_file.py:68: Initiat

Converting: /content/drive/MyDrive/hust_docs/01_3%20HD%20hoc%20chuyen%20tiep%20ky%20su%20180%20TC_Final.pdf


[INFO] 2026-01-03 08:20:35,589 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:20:35,590 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:20:35,631 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:20:35,632 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:20:35,866 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:20:35,867 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:20:35,871 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-03 08:20:35,872 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-0

Converting: /content/drive/MyDrive/hust_docs/02_%20Th%E1%BB%A7%20th%E1%BB%A5c_Bi%E1%BB%83u%20m%E1%BA%ABu%20(k%C3%A8m%20Quy%20ch%E1%BA%BF%20%C4%90T).pdf


...

Converting: /content/drive/MyDrive/hust_docs/QD%20ban%20hanh%20QD%20to%20chuc%20thi%20Truc%20tuyen.pdf


[INFO] 2026-01-03 08:25:21,832 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:25:21,833 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:25:21,879 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:25:21,880 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:25:22,121 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:25:22,123 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:25:22,129 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-03 08:25:22,130 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-0

Converting: /content/drive/MyDrive/hust_docs/QD_ngoai_ngu_tu_K68_CQ_final.pdf


[INFO] 2026-01-03 08:25:40,131 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:25:40,134 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:25:40,207 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:25:40,209 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-03 08:25:40,600 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-03 08:25:40,601 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-03 08:25:40,605 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-03 08:25:40,606 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-01-0

Total blocks: 681
Indexed 4/681
Indexed 44/681
Indexed 84/681
Indexed 124/681
Indexed 164/681
Indexed 204/681
Indexed 244/681
Indexed 284/681
Indexed 324/681
Indexed 364/681
Indexed 404/681
Indexed 444/681
Indexed 484/681
Indexed 524/681
Indexed 564/681
Indexed 604/681
Indexed 644/681
Indexed 681/681
Index and metadata saved


In [ ]:
def retrieve(query, embedder, index, all_docs, top_k=5):
    query_emb = embedder.encode(
        [query],
        normalize_embeddings=True
    )

    scores, indices = index.search(
        np.array(query_emb, dtype="float32"),
        top_k
    )

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:
            continue
        results.append({
            "score": float(score),
            "text": all_docs[idx]["text"],
            "metadata": all_docs[idx]["metadata"]
        })

    return results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install faiss-cpu -q

In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

index = faiss.read_index(f"{OUT_DIR}/rag.index")

with open(f"{OUT_DIR}/metadata.json", "r", encoding="utf-8") as f:
    docs = json.load(f)

print("Index size :", index.ntotal)
print("Docs count :", len(docs))

assert index.ntotal == len(docs)

embedder = SentenceTransformer(EMBED_MODEL)

Index size : 681
Docs count : 681


In [ ]:
query = "Cần chứng chỉ tiếng anh gì để được miễn học phần tiếng anh"

results = retrieve(
    query=query,
    embedder=embedder,
    index=index,
    all_docs=docs,
    top_k=40
)

for i, r in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print(f"Score   : {r['score']:.4f}")
    print(f"Section : {r['metadata']['section']}")
    print(f"Type    : {r['metadata']['type']}")
    print(f"Source  : {r['metadata']['source']}")
    print(r["text"])



--- Result 1 ---
Score   : 0.7023
Section : ROOT
Type    : table
Source  : QD_ngoai_ngu_tu_K68_CQ_final.pdf
Bảng 6.1 Danh mục các học phần tiếng Anh yêu cầu

|TT |Mã học phần| Tên học phần            |Thời lượng| Tính tín chỉ trong CTĐT| Học phần được miễn/cần học theo chứng chỉ đạt được| Học phần được miễn/cần học theo chứng chỉ đạt được| Học phần được miễn/cần học theo chứng chỉ đạt được| Học phần được miễn/cần học theo chứng chỉ đạt được| Học phần được miễn/cần học theo chứng chỉ đạt được|
|------|---------------|------------------------------|--------------|---------------------------|------------------------------------------------------|------------------------------------------------------|------------------------------------------------------|------------------------------------------------------|------------------------------------------------------|
|      |               |                              |              |                           | [1]                         

In [ ]:
from google.colab import runtime
runtime.unassign()